In [2]:
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample

from datetime import datetime, timedelta

## Preprocessing

In [3]:
df = pd.read_csv("E:/Capstone-Retail/data_dsmarket/new_files/sales_calendar_prices.csv") #Marina Windows
#df = pd.read_csv("/Users/marinaramiropareta/Desktop/Capstone_Retail/data_dsmarket/new_files/sales_calendar_prices.csv") #Marina Mac

df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
0,2011-10,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-07,NaN,NaN,NaN
1,2011-10,ACCESORIES_1_001_BOS_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,2011-03-07,NaN,NaN,NaN
2,2011-10,ACCESORIES_1_001_BOS_3,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,2011-03-07,NaN,NaN,NaN
3,2011-10,ACCESORIES_1_001_NYC_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-03-07,NaN,NaN,NaN
4,2011-10,ACCESORIES_1_001_NYC_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,2011-03-07,NaN,NaN,NaN


In [5]:
df.shape

(8354260, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   year_week   object 
 1   id          object 
 2   units       int64  
 3   item        object 
 4   category    object 
 5   department  object 
 6   store       object 
 7   store_code  object 
 8   region      object 
 9   date        object 
 10  sell_price  float64
 11  year        float64
 12  week        float64
dtypes: float64(3), int64(1), object(9)
memory usage: 828.6+ MB


* Tenemos 'date' como object, habrá que pasarlo a fecha.
* Podemos pasar 'year' y 'week' a integers ✅, y luego también a un formato de fecha.
* Podemos generar también una columna de 'month' y de 'day', para acabar de tener todos los datos. ✅

In [7]:
# Transformar "date" de object -> datetime64[ns]
df["date"] = pd.to_datetime(df["date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 13 columns):
 #   Column      Dtype         
---  ------      -----         
 0   year_week   object        
 1   id          object        
 2   units       int64         
 3   item        object        
 4   category    object        
 5   department  object        
 6   store       object        
 7   store_code  object        
 8   region      object        
 9   date        datetime64[ns]
 10  sell_price  float64       
 11  year        float64       
 12  week        float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(8)
memory usage: 828.6+ MB


In [8]:
df[df.duplicated()]

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week


In [9]:
df.isnull().sum()

year_week           0
id                  0
units               0
item                0
category            0
department          0
store               0
store_code          0
region              0
date                0
sell_price    3008732
year          3008732
week          3008732
dtype: int64

In [10]:
df[df['sell_price'].isnull()].head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
0,2011-10,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-07,NaN,NaN,NaN
1,2011-10,ACCESORIES_1_001_BOS_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,2011-03-07,NaN,NaN,NaN
2,2011-10,ACCESORIES_1_001_BOS_3,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,2011-03-07,NaN,NaN,NaN
3,2011-10,ACCESORIES_1_001_NYC_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-03-07,NaN,NaN,NaN
4,2011-10,ACCESORIES_1_001_NYC_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,2011-03-07,NaN,NaN,NaN


* Para imputar los nulos de "sell_price" hay dos estrategias:

    1. Eliminar directamente las filas que contengan NaNs. ❌

In [11]:
# Porcentaje de datos que eliminaríamos:
3008732 / 8354260 * 100

36.01434477739501

In [12]:
# Igualemnte nos quedaríamos con:
8354260 - 3008732

5345528

In [13]:
df_without_nulls = df.dropna()
df_without_nulls.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
10,2011-10,ACCESORIES_1_002_BOS_1,4,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-07,5.28,2011.0,10.0
12,2011-10,ACCESORIES_1_002_BOS_3,2,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,2011-03-07,5.28,2011.0,10.0
14,2011-10,ACCESORIES_1_002_NYC_2,1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,2011-03-07,5.28,2011.0,10.0
19,2011-10,ACCESORIES_1_002_PHI_3,3,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Queen_Village,PHI_3,Philadelphia,2011-03-07,5.28,2011.0,10.0
30,2011-10,ACCESORIES_1_004_BOS_1,4,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-07,5.77,2011.0,10.0


In [14]:
df_without_nulls.isnull().sum()

year_week     0
id            0
units         0
item          0
category      0
department    0
store         0
store_code    0
region        0
date          0
sell_price    0
year          0
week          0
dtype: int64

In [15]:
df_without_nulls.shape

(5345528, 13)

In [16]:
df_without_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5345528 entries, 10 to 8110339
Data columns (total 13 columns):
 #   Column      Dtype         
---  ------      -----         
 0   year_week   object        
 1   id          object        
 2   units       int64         
 3   item        object        
 4   category    object        
 5   department  object        
 6   store       object        
 7   store_code  object        
 8   region      object        
 9   date        datetime64[ns]
 10  sell_price  float64       
 11  year        float64       
 12  week        float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(8)
memory usage: 571.0+ MB


2. Imputar los valores NaN en la columna 'sell_price' con los valores más recientes de la misma colmna, teniendo en cuenta el mismo 'id'. Vamosa  usar el método **bfilll()** junto con **groupby()** para realizar el relleno hacia atrás por grupos. Primero vamos a asegurarnos que los datos estén ordenado antes de realizar el bakfill. ✅

In [17]:
df_filled = df.sort_values(['id','date'])
df_filled.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
914700,2011-4,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-29,NaN,NaN,NaN
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,NaN,NaN,NaN
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,NaN,NaN,NaN
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,NaN,NaN,NaN
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,NaN,NaN,NaN


In [18]:
df_filled['sell_price'] = df_filled.groupby(['item'])['sell_price'].fillna(method='ffill').fillna(method='bfill')
df_filled.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
914700,2011-4,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-29,12.74,NaN,NaN
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,12.74,NaN,NaN
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,12.74,NaN,NaN
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,12.74,NaN,NaN
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,12.74,NaN,NaN


In [19]:
df_filled["week"] = df_filled["date"].dt.isocalendar().week.astype(str)
df_filled["year"] = df_filled["date"].dt.isocalendar().year.astype(str)
df_filled.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
914700,2011-4,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-29,12.74,2011,4
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,12.74,2011,5
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,12.74,2011,6
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,12.74,2011,7
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,12.74,2011,8


In [20]:
# Usamos el df_filled
df_final = df_filled.copy()
df_final.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
914700,2011-4,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-29,12.74,2011,4
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,12.74,2011,5
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,12.74,2011,6
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,12.74,2011,7
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,12.74,2011,8


In [21]:
df_final['year'] = df_final["year"].apply(int)
df_final["month_int"] = df_final["date"].dt.month.astype(str)
df_final["month"] = df_final["date"].dt.strftime("%B")
df_final['week'] = df_final["week"].apply(int)
df_final["day"] = df_final["date"].dt.day.astype(str)

df_final.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week,month_int,month,day
914700,2011-4,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-29,12.74,2011,4,1,January,29
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,12.74,2011,5,1,January,31
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,12.74,2011,6,2,February,7
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,12.74,2011,7,2,February,14
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,12.74,2011,8,2,February,21


* Vamos a eliminar los registros de la 1ª semana, ya que vimos (en "1_aux_db") que estaban incompletos.

In [22]:
df_final = df_final[df_final["year_week"] != "2011-4"]
df_final

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week,month_int,month,day
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,12.74,2011,5,1,January,31
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,12.74,2011,6,2,February,7
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,12.74,2011,7,2,February,14
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,12.74,2011,8,2,February,21
1463520,2011-9,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-28,12.74,2011,9,2,February,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7988379,2016-12,SUPERMARKET_3_827_PHI_3,17,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-03-21,1.20,2016,12,3,March,21
8018869,2016-13,SUPERMARKET_3_827_PHI_3,22,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-03-28,1.20,2016,13,3,March,28
8049359,2016-14,SUPERMARKET_3_827_PHI_3,8,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-04,1.20,2016,14,4,April,4
8079849,2016-15,SUPERMARKET_3_827_PHI_3,0,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-11,1.20,2016,15,4,April,11


## Feature engineering - General features

In [23]:
df_final['revenue'] = df_final['units'] * df_final['sell_price']
df_final.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week,month_int,month,day,revenue
1250090,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,12.74,2011,5,1,January,31,0.0
1372050,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,12.74,2011,6,2,February,7,0.0
1402540,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,12.74,2011,7,2,February,14,0.0
1433030,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,12.74,2011,8,2,February,21,0.0
1463520,2011-9,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-28,12.74,2011,9,2,February,28,0.0


### Campaña de eventos (extendidos)

In [24]:
calendar = pd.read_csv("E:/Capstone-Retail/data_dsmarket/new_files/daily_calendar_with_events_mod.csv") #Marina Windows
#calendar = pd.read_csv("/Users/marinaramiropareta/Desktop/Capstone_Retail/data_dsmarket/new_files/daily_calendar_with_events_mod.csv")
calendar.head()

,Unnamed: 0,date,weekday,weekday_int,d,event,week,year,year_week
0,0,2011-01-29,Saturday,1,d_1,0,4,2011,2011-4
1,1,2011-01-30,Sunday,2,d_2,0,4,2011,2011-4
2,2,2011-01-31,Monday,3,d_3,0,5,2011,2011-5
3,3,2011-02-01,Tuesday,4,d_4,0,5,2011,2011-5
4,4,2011-02-02,Wednesday,5,d_5,0,5,2011,2011-5


In [25]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1913 non-null   int64 
 1   date         1913 non-null   object
 2   weekday      1913 non-null   object
 3   weekday_int  1913 non-null   int64 
 4   d            1913 non-null   object
 5   event        1913 non-null   object
 6   week         1913 non-null   int64 
 7   year         1913 non-null   int64 
 8   year_week    1913 non-null   object
dtypes: int64(4), object(5)
memory usage: 134.6+ KB


In [26]:
# Transformar "date" de object -> datetime64[ns]
calendar["date"] = pd.to_datetime(calendar["date"])

In [27]:
calendar.drop(columns = ['Unnamed: 0'], inplace=True)
calendar.head()

,date,weekday,weekday_int,d,event,week,year,year_week
0,2011-01-29,Saturday,1,d_1,0,4,2011,2011-4
1,2011-01-30,Sunday,2,d_2,0,4,2011,2011-4
2,2011-01-31,Monday,3,d_3,0,5,2011,2011-5
3,2011-02-01,Tuesday,4,d_4,0,5,2011,2011-5
4,2011-02-02,Wednesday,5,d_5,0,5,2011,2011-5


In [28]:
calendar["event"].value_counts()

0                 1887
SuperBowl            6
Ramadan starts       5
Thanksgiving         5
NewYear              5
Easter               5
Name: event, dtype: int64

In [29]:
# Vamos a generar una nueva columna para cada uno de los eventos, apra tratarlos de manera inidividual
calendar["event_Ramadan"] = np.where(calendar["event"] == "Ramadan starts", 1, 0)
calendar["event_SuperBowl"] = np.where(calendar["event"] == "SuperBowl", 1, 0)
calendar["event_Thanksgiving"] = np.where(calendar["event"] == "Thanksgiving", 1, 0)
calendar["event_NewYear"] = np.where(calendar["event"] == "NewYear", 1, 0)
calendar["event_Easter"] = np.where(calendar["event"] == "Easter", 1, 0)

calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1913 non-null   datetime64[ns]
 1   weekday             1913 non-null   object        
 2   weekday_int         1913 non-null   int64         
 3   d                   1913 non-null   object        
 4   event               1913 non-null   object        
 5   week                1913 non-null   int64         
 6   year                1913 non-null   int64         
 7   year_week           1913 non-null   object        
 8   event_Ramadan       1913 non-null   int32         
 9   event_SuperBowl     1913 non-null   int32         
 10  event_Thanksgiving  1913 non-null   int32         
 11  event_NewYear       1913 non-null   int32         
 12  event_Easter        1913 non-null   int32         
dtypes: datetime64[ns](1), int32(5), int64(3), object

* Vamos a tener en cuenta que la campaña de ventas por los eventos de (SuperBowl, Thanksgiving, NewYear y Easter) empieza 1 semana antes para cada uno de los eventos. ✅

In [30]:
def extend_events_previous_days(df, columns, previous_days):
    # Asegúrate de que los datos estén ordenados por fecha
    df = df.sort_values(by='date')

    # Crea un DataFrame temporal que marque los eventos y los días anteriores como eventos
    df_temp = df.copy()

    for column in columns:
        # Crea una columna extendida para el evento actual
        df_temp[column + '_extended'] = df_temp[column]

        # Extiende los eventos hacia los días anteriores
        for i in range(1, previous_days):
            shifted = df[column].shift(i).fillna(0).astype(int)
            df_temp[column + '_extended'] = df_temp[column + '_extended'] | shifted

        # Reemplaza la columna original con la columna extendida
        df[column] = df_temp[column + '_extended']

    # Limpia
    del df_temp

    return df

In [31]:
calendar["event_SuperBowl"].value_counts()

0    1907
1       6
Name: event_SuperBowl, dtype: int64

In [32]:
calendar = extend_events_previous_days(df = calendar, 
                                       columns =["event_SuperBowl",                                                
                                                 "event_Thanksgiving", 
                                                 "event_NewYear", 
                                                 "event_Easter"],
                                       previous_days = 7)

In [33]:
calendar["event_SuperBowl"].value_counts()

0    1871
1      42
Name: event_SuperBowl, dtype: int64

In [34]:
# Verificamos que realmente se han añadido 7 días más a cada evento:
6*7

42

* En el caso del Ramadán vamos a tener en cuenta que dura un mes, así que vamos a tener en cuenta el inicio +30 días después de que empiece.

In [35]:
def extend_events_posterior_days(df, columns, posterior_days):
    # Asegúrate de que los datos estén ordenados por fecha
    df = df.sort_values(by='date')

    # Crea un DataFrame temporal que marque los eventos y los días posteriores como eventos
    df_temp = df.copy()

    for column in columns:
        # Crea una columna extendida para el evento actual
        df_temp[column + '_extended'] = df_temp[column]

        # Extiende los eventos hacia los días posteriores
        for i in range(1, posterior_days):
            shifted = df[column].shift(-i).fillna(0).astype(int)
            df_temp[column + '_extended'] = df_temp[column + '_extended'] | shifted

        # Reemplaza la columna original con la columna extendida
        df[column] = df_temp[column + '_extended']

    # Limpia
    del df_temp

    return df

In [36]:
calendar["event_Ramadan"].value_counts()

0    1908
1       5
Name: event_Ramadan, dtype: int64

In [37]:
calendar = extend_events_posterior_days(df = calendar, 
                                        columns = ["event_Ramadan"],
                                        posterior_days = 30)

In [38]:
calendar["event_Ramadan"].value_counts()

0    1763
1     150
Name: event_Ramadan, dtype: int64

In [39]:
# Verificamos que realmente se han añadido 7 días más a cada evento:
5*30

150

In [40]:
# Ahora vamos a generar un nuevo dummy con todos los eventos extendido, para hacerlo más general
calendar['event_any'] = np.where(calendar[['event_SuperBowl', 
                                           'event_Thanksgiving', 
                                           'event_Easter',
                                           'event_NewYear',
                                           'event_Ramadan']].any(axis=1), 1, 0)
calendar['event_any'].value_counts()

0    1616
1     297
Name: event_any, dtype: int64

In [41]:
calendar.drop(columns="event", inplace=True)

In [42]:
calendar.head()

,date,weekday,weekday_int,d,week,year,year_week,event_Ramadan,event_SuperBowl,event_Thanksgiving,event_NewYear,event_Easter,event_any
0,2011-01-29,Saturday,1,d_1,4,2011,2011-4,0,0,0,0,0,0
1,2011-01-30,Sunday,2,d_2,4,2011,2011-4,0,0,0,0,0,0
2,2011-01-31,Monday,3,d_3,5,2011,2011-5,0,0,0,0,0,0
3,2011-02-01,Tuesday,4,d_4,5,2011,2011-5,0,0,0,0,0,0
4,2011-02-02,Wednesday,5,d_5,5,2011,2011-5,0,0,0,0,0,0


In [43]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1913 entries, 0 to 1912
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1913 non-null   datetime64[ns]
 1   weekday             1913 non-null   object        
 2   weekday_int         1913 non-null   int64         
 3   d                   1913 non-null   object        
 4   week                1913 non-null   int64         
 5   year                1913 non-null   int64         
 6   year_week           1913 non-null   object        
 7   event_Ramadan       1913 non-null   int32         
 8   event_SuperBowl     1913 non-null   int32         
 9   event_Thanksgiving  1913 non-null   int32         
 10  event_NewYear       1913 non-null   int32         
 11  event_Easter        1913 non-null   int32         
 12  event_any           1913 non-null   int32         
dtypes: datetime64[ns](1), int32(6), int64(3), object

In [44]:
calendar.to_csv("E:/Capstone-Retail/data_dsmarket/new_files/daily_calendar_with_events_mod_extended.csv") #Marina Windows
#calendar.to_csv("/Users/marinaramiropareta/Desktop/Capstone_Retail/data_dsmarket/new_files/daily_calendar_with_events_mod_extended.csv") #Marina Mac

In [45]:
calendar.drop(columns=["d", 'week', 'year', 'year_week'], inplace=True)

In [46]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1913 entries, 0 to 1912
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1913 non-null   datetime64[ns]
 1   weekday             1913 non-null   object        
 2   weekday_int         1913 non-null   int64         
 3   event_Ramadan       1913 non-null   int32         
 4   event_SuperBowl     1913 non-null   int32         
 5   event_Thanksgiving  1913 non-null   int32         
 6   event_NewYear       1913 non-null   int32         
 7   event_Easter        1913 non-null   int32         
 8   event_any           1913 non-null   int32         
dtypes: datetime64[ns](1), int32(6), int64(1), object(1)
memory usage: 104.6+ KB


In [47]:
df_final = pd.merge(
    left = df_final,
    right = calendar,
    how = "left",
    on = ["date"]
)

df_final.head()

,year_week,id,units,item,category,department,store,store_code,region,date,...,day,revenue,weekday,weekday_int,event_Ramadan,event_SuperBowl,event_Thanksgiving,event_NewYear,event_Easter,event_any
0,2011-5,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,...,31,0.0,Monday,3,0,0,0,0,0,0
1,2011-6,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-07,...,7,0.0,Monday,3,0,1,0,0,0,1
2,2011-7,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-14,...,14,0.0,Monday,3,0,0,0,0,0,0
3,2011-8,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-21,...,21,0.0,Monday,3,0,0,0,0,0,0
4,2011-9,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-28,...,28,0.0,Monday,3,0,0,0,0,0,0


In [48]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323770 entries, 0 to 8323769
Data columns (total 25 columns):
 #   Column              Dtype         
---  ------              -----         
 0   year_week           object        
 1   id                  object        
 2   units               int64         
 3   item                object        
 4   category            object        
 5   department          object        
 6   store               object        
 7   store_code          object        
 8   region              object        
 9   date                datetime64[ns]
 10  sell_price          float64       
 11  year                int64         
 12  week                int64         
 13  month_int           object        
 14  month               object        
 15  day                 object        
 16  revenue             float64       
 17  weekday             object        
 18  weekday_int         int64         
 19  event_Ramadan       int32         
 20  ev

In [49]:
# Lista de nombres de columnas en el orden deseado
column_order = ['date',
                'year',
                'month',
                'month_int',
                'week',
                'year_week',
                'day',
                'weekday',
                'weekday_int',
                'event_any',
                'event_SuperBowl',
                'event_Thanksgiving',
                'event_Ramadan',
                'event_NewYear',
                'event_Easter', 
                'id',
                'item',
                'category',
                'department',
                'store', 
                'store_code', 
                'region',
                'units',
                'sell_price',
                'revenue']

df_final = df_final.reindex(columns=column_order)
df_final.head()

,date,year,month,month_int,week,year_week,day,weekday,weekday_int,event_any,...,id,item,category,department,store,store_code,region,units,sell_price,revenue
0,2011-01-31,2011,January,1,5,2011-5,31,Monday,3,0,...,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0
1,2011-02-07,2011,February,2,6,2011-6,7,Monday,3,1,...,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0
2,2011-02-14,2011,February,2,7,2011-7,14,Monday,3,0,...,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0
3,2011-02-21,2011,February,2,8,2011-8,21,Monday,3,0,...,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0
4,2011-02-28,2011,February,2,9,2011-9,28,Monday,3,0,...,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0


### Otras campañas:

In [50]:
# Creamos una columna que tenga las ventas de verano:
df_final['summer_sales'] = np.where(df_final["month"].isin(["June","July", "August"]), df_final.units, 0)

# Creamos una columna que tenga las ventas de Navidad:
df_final['christmas_sales'] = np.where(df_final["month"].isin(["November","December"]), df_final.units, 0)

df_final.head()

,date,year,month,month_int,week,year_week,day,weekday,weekday_int,event_any,...,category,department,store,store_code,region,units,sell_price,revenue,summer_sales,christmas_sales
0,2011-01-31,2011,January,1,5,2011-5,31,Monday,3,0,...,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0,0,0
1,2011-02-07,2011,February,2,6,2011-6,7,Monday,3,1,...,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0,0,0
2,2011-02-14,2011,February,2,7,2011-7,14,Monday,3,0,...,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0,0,0
3,2011-02-21,2011,February,2,8,2011-8,21,Monday,3,0,...,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0,0,0
4,2011-02-28,2011,February,2,9,2011-9,28,Monday,3,0,...,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,12.74,0.0,0,0


In [51]:
# Creamos una columna que tenga las ventas en los eventos:
df_final["event_sales"] = np.where(df_final["event_any"] == 1, df_final.units, 0)

df_final["event_SuperBowl_sales"] = np.where(df_final["event_SuperBowl"] == 1, df_final.units, 0)
df_final["event_Thanksgiving_sales"] = np.where(df_final["event_Thanksgiving"] == 1, df_final.units, 0)
df_final["event_Ramadan_sales"] = np.where(df_final["event_Ramadan"] == 1, df_final.units, 0)
df_final["event_NewYear_sales"] = np.where(df_final["event_NewYear"] == 1, df_final.units, 0)
df_final["event_Easter_sales"] = np.where(df_final["event_Easter"] == 1, df_final.units, 0)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323770 entries, 0 to 8323769
Data columns (total 33 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   date                      datetime64[ns]
 1   year                      int64         
 2   month                     object        
 3   month_int                 object        
 4   week                      int64         
 5   year_week                 object        
 6   day                       object        
 7   weekday                   object        
 8   weekday_int               int64         
 9   event_any                 int32         
 10  event_SuperBowl           int32         
 11  event_Thanksgiving        int32         
 12  event_Ramadan             int32         
 13  event_NewYear             int32         
 14  event_Easter              int32         
 15  id                        object        
 16  item                      object        
 17  category

### Ventas por año

In [52]:
df_final["sales_2011"] = np.where((df_final.year == 2011), df_final.units, 0) # Sabemos que no está completo
df_final["sales_2012"] = np.where((df_final.year == 2012), df_final.units, 0)
df_final["sales_2013"] = np.where((df_final.year == 2013), df_final.units, 0)
df_final["sales_2014"] = np.where((df_final.year == 2014), df_final.units, 0)
df_final["sales_2015"] = np.where((df_final.year == 2015), df_final.units, 0)
df_final["sales_2016"] = np.where((df_final.year == 2016), df_final.units, 0) # Sabemos que no está completo

df_final.sample(5)

,date,year,month,month_int,week,year_week,day,weekday,weekday_int,event_any,...,event_Thanksgiving_sales,event_Ramadan_sales,event_NewYear_sales,event_Easter_sales,sales_2011,sales_2012,sales_2013,sales_2014,sales_2015,sales_2016
3088066,2014-03-17,2014,March,3,12,2014-12,17,Monday,3,0,...,0,0,0,0,0,0,0,0,0,0
6820803,2014-05-12,2014,May,5,20,2014-20,12,Monday,3,0,...,0,0,0,0,0,0,0,0,0,0
1611579,2012-03-26,2012,March,3,13,2012-13,26,Monday,3,0,...,0,0,0,0,0,0,0,0,0,0
1828059,2012-01-23,2012,January,1,4,2012-4,23,Monday,3,0,...,0,0,0,0,0,0,0,0,0,0
7419639,2011-12-12,2011,December,12,50,2011-50,12,Monday,3,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# Obtener la fecha máxima de 2016
max_date = df_final['date'].max()
max_date

Timestamp('2016-04-18 00:00:00')

In [54]:
# Crear una seleccion para las fechas hasta el 18 de abril de 2015
df_final["sales_2015_rel"] = np.where(((df_final.year == 2015) & (df_final.date <= '2015-04-18')), df_final.units, 0)

In [55]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323770 entries, 0 to 8323769
Data columns (total 40 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   date                      datetime64[ns]
 1   year                      int64         
 2   month                     object        
 3   month_int                 object        
 4   week                      int64         
 5   year_week                 object        
 6   day                       object        
 7   weekday                   object        
 8   weekday_int               int64         
 9   event_any                 int32         
 10  event_SuperBowl           int32         
 11  event_Thanksgiving        int32         
 12  event_Ramadan             int32         
 13  event_NewYear             int32         
 14  event_Easter              int32         
 15  id                        object        
 16  item                      object        
 17  category

In [56]:
df_final.to_csv("E:/Capstone-Retail/data_dsmarket/new_files/df_forecast_1.csv")  #Marina Windows
#df_final.to_csv("/Users/marinaramiropareta/Desktop/Capstone_Retail/data_dsmarket/new_files/df_forecast_1.csv") #Marina Mac